In [11]:
!pip install nltk

In [12]:
!pip install pymorphy3

#**1. Импорт библиотек и загрузка данных NLTK**

In [13]:
import nltk
import numpy as np
import re
from collections import Counter
from pymorphy3 import MorphAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#**2. Класс SimpleTransformer (простая трансформерная модель)**

In [14]:
class SimpleTransformer:
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim

        self.W_embed = np.random.randn(self.vocab_size, self.embed_dim) * 0.01
        self.W_hidden = np.random.randn(self.embed_dim, self.hidden_dim) * 0.01
        self.W_out = np.random.randn(self.hidden_dim, self.vocab_size) * 0.01
        self.b_hidden = np.zeros((1, self.hidden_dim))
        self.b_out = np.zeros((1, self.vocab_size))

    def forward(self, X):
        # Процесс пропуска через сеть
        embed = np.dot(X, self.W_embed)
        hidden = np.tanh(np.dot(embed, self.W_hidden) + self.b_hidden)
        output = np.dot(hidden, self.W_out) + self.b_out
        probs = self.softmax(output)
        return probs

    def softmax(self, x):
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def train_step(self, X, y):
        # Обучающий шаг
        probs = self.forward(X)
        loss = -np.log(probs[0, y] + 1e-9)
        grad_output = probs
        grad_output[0, y] -= 1
        grad_W_out = np.outer(grad_output, X)

        grad_hidden = np.dot(grad_output, self.W_out.T) * (1 - np.power(X, 2))
        grad_W_hidden = np.outer(grad_hidden, X)

        grad_W_embed = np.outer(grad_hidden, X)
        return loss

    def predict(self, X):
        probs = self.forward(X)
        return np.argmax(probs)


#**3. Класс TextNeuralNetwork (нейросеть для обработки текста)**

In [15]:
class TextNeuralNetwork:
    def __init__(self, hidden_size=128, seq_length=3, learning_rate=0.01):
        """
        Инициализация нейронной сети для текста:
        - hidden_size: количество нейронов в скрытом слое (увеличено до 128)
        - seq_length: длина последовательности для предсказания
        - learning_rate: скорость обучения
        """
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        self.word_to_idx = {}
        self.idx_to_word = {}
        self.vocab_size = 0
        self.loss_history = []
    # Альтернатива для русского языка (вместо NLTK)


    def preprocess_text(self, text):
        morph = MorphAnalyzer()
        tokens = re.findall(r'\b[а-яё]+\b', text.lower())
        stop_words = {'и', 'в', 'на', 'с', 'по', 'для', 'не', 'а', 'но', 'или'}  # можно расширить
        tokens = [morph.parse(word)[0].normal_form for word in tokens if word not in stop_words]

        word_counts = Counter(tokens)
        vocab = sorted(word_counts, key=word_counts.get, reverse=True)

        self.word_to_idx = {word: idx for idx, word in enumerate(vocab)}
        self.idx_to_word = {idx: word for idx, word in enumerate(vocab)}
        self.vocab_size = len(vocab)

        return tokens

    def create_training_data(self, tokens):
        """Создание обучающих данных (X, y)."""
        X = []
        y = []

        for i in range(len(tokens) - self.seq_length):
            seq = tokens[i:i + self.seq_length]
            next_word = tokens[i + self.seq_length]

            X.append([self.word_to_idx[word] for word in seq])
            y.append(self.word_to_idx[next_word])

        return np.array(X), np.array(y)

    def init_weights(self):
        """Инициализация весов сети."""
        self.W1 = np.random.randn(self.vocab_size, self.hidden_size) * 0.01
        self.b1 = np.zeros((1, self.hidden_size))
        self.W2 = np.random.randn(self.hidden_size, self.vocab_size) * 0.01
        self.b2 = np.zeros((1, self.vocab_size))

    def softmax(self, x):
        """Функция активации Softmax."""
        exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=1, keepdims=True)

    def forward(self, X):
        """Прямое распространение."""
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = np.tanh(self.z1)
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.a2 = self.softmax(self.z2)
        return self.a2

    def compute_loss(self, y_true, y_pred):
        """Вычисление кросс-энтропийной потери."""
        m = y_true.shape[0]
        log_probs = -np.log(y_pred[range(m), y_true] + 1e-9)  # Добавлен эпсилон для численной стабильности
        loss = np.sum(log_probs) / m
        return loss

    def backward(self, X, y_true, y_pred):
        """Обратное распространение."""
        m = y_true.shape[0]

        # Градиент выходного слоя
        grad_z2 = y_pred
        grad_z2[range(m), y_true] -= 1
        grad_z2 /= m

        # Градиенты параметров
        grad_W2 = np.dot(self.a1.T, grad_z2)
        grad_b2 = np.sum(grad_z2, axis=0, keepdims=True)

        # Градиент скрытого слоя
        grad_a1 = np.dot(grad_z2, self.W2.T)
        grad_z1 = grad_a1 * (1 - np.power(self.a1, 2))  # Производная tanh

        grad_W1 = np.dot(X.T, grad_z1)
        grad_b1 = np.sum(grad_z1, axis=0, keepdims=True)

        return grad_W1, grad_b1, grad_W2, grad_b2

    def train(self, text, epochs=10000, verbose=True):
        """Обучение сети на тексте (увеличено количество эпох до 500)."""
        tokens = self.preprocess_text(text)
        X, y = self.create_training_data(tokens)
        self.init_weights()

        # Преобразование X в one-hot encoding
        X_onehot = np.zeros((len(X), self.vocab_size))
        for i, seq in enumerate(X):
            for word_idx in seq:
                X_onehot[i, word_idx] = 1

        for epoch in range(epochs):
            # Прямое распространение
            y_pred = self.forward(X_onehot)

            # Вычисление потерь
            loss = self.compute_loss(y, y_pred)
            self.loss_history.append(loss)

            # Обратное распространение
            grad_W1, grad_b1, grad_W2, grad_b2 = self.backward(X_onehot, y, y_pred)

            # Обновление весов
            self.W1 -= self.learning_rate * grad_W1
            self.b1 -= self.learning_rate * grad_b1
            self.W2 -= self.learning_rate * grad_W2
            self.b2 -= self.learning_rate * grad_b2

            if verbose and epoch % 50 == 0:  # Вывод каждые 50 эпох
                print(f"Epoch {epoch}, Loss: {loss:.4f}")

    def predict_next_word(self, input_seq):
        """Предсказание следующего слова по последовательности с обработкой неизвестных слов."""
        seq_indices = []
        for word in input_seq:
            if word in self.word_to_idx:
                seq_indices.append(self.word_to_idx[word])
            else:
                seq_indices.append(0)  # Используем индекс 0 для неизвестных слов

        X = np.zeros((1, self.vocab_size))
        for idx in seq_indices:
            X[0, idx] = 1

        y_pred = self.forward(X)
        next_word_idx = np.argmax(y_pred)
        return self.idx_to_word[next_word_idx]

    def evaluate(self, orders, result):
        """Оценка модели на тестовых данных."""
        correct = 0
        for i, order in enumerate(orders):
            input_seq = re.findall(r'\b\w+\b', order.lower())
            if len(input_seq) < self.seq_length:
                print(f"Фраза '{order}' слишком короткая для предсказания")
                continue

            input_seq = input_seq[-self.seq_length:]  # Берем последние seq_length слов
            try:
                predicted = self.predict_next_word(input_seq)
                if predicted == result[i]:
                    correct += 1
                print(f"'{order}' -> предсказано: '{predicted}', ожидалось: '{result[i]}'")
            except Exception as e:
                print(f"Ошибка при обработке фразы '{order}': {str(e)}")

        accuracy = correct / len(orders)
        print(f"\nTest Accuracy: {accuracy * 100:.2f}%")

#4**. Тренировочные данные (входной текст и тестовые примеры)**

In [16]:
# Новый текст
text = 'На берегу моря стояла девушка. Ветры играли с её волосами, а волны мягко омывали песок. Она смотрела вдаль, где небо сливалось с горизонтом, поглощая всю тревогу и усталость. В её душе царила тишина, она будто почувствовала каждый момент. Эти мгновения были для неё самыми ценными, ведь на этом пляже она всегда чувствовала себя свободной. Вспоминая прошлые дни, она думала о том, как изменилась её жизнь. Старая работа, старые друзья, и эти неведомые горизонты, которые стали её путеводной звездой.'

# Новые тестовые данные
orders = [
    'Девушка стояла на пляже',
    'Волны омывали берег',
    'Она смотрела в даль',
    'Морской воздух был',
    'Тишина вокруг неё',
    'Она думала о жизни',
    'Свобода ощущалась',
    'Прошлые дни пролетели'
]

result = [
    'одинокая', 'гладкими', 'мечтая', 'освежающий', 'завораживающая', 'грустные', 'необъятная', 'воспоминания'
]

# Обучение и оценка
model = TextNeuralNetwork()
model.train(text, epochs=10000)
model.evaluate(orders, result)

Epoch 0, Loss: 4.0775
Epoch 50, Loss: 4.0735
Epoch 100, Loss: 4.0695
Epoch 150, Loss: 4.0656
Epoch 200, Loss: 4.0618
Epoch 250, Loss: 4.0580
Epoch 300, Loss: 4.0543
Epoch 350, Loss: 4.0506
Epoch 400, Loss: 4.0470
Epoch 450, Loss: 4.0434
Epoch 500, Loss: 4.0399
Epoch 550, Loss: 4.0364
Epoch 600, Loss: 4.0330
Epoch 650, Loss: 4.0296
Epoch 700, Loss: 4.0262
Epoch 750, Loss: 4.0229
Epoch 800, Loss: 4.0196
Epoch 850, Loss: 4.0164
Epoch 900, Loss: 4.0132
Epoch 950, Loss: 4.0100
Epoch 1000, Loss: 4.0069
Epoch 1050, Loss: 4.0037
Epoch 1100, Loss: 4.0006
Epoch 1150, Loss: 3.9975
Epoch 1200, Loss: 3.9945
Epoch 1250, Loss: 3.9914
Epoch 1300, Loss: 3.9884
Epoch 1350, Loss: 3.9854
Epoch 1400, Loss: 3.9824
Epoch 1450, Loss: 3.9794
Epoch 1500, Loss: 3.9763
Epoch 1550, Loss: 3.9733
Epoch 1600, Loss: 3.9703
Epoch 1650, Loss: 3.9673
Epoch 1700, Loss: 3.9642
Epoch 1750, Loss: 3.9611
Epoch 1800, Loss: 3.9581
Epoch 1850, Loss: 3.9549
Epoch 1900, Loss: 3.9518
Epoch 1950, Loss: 3.9486
Epoch 2000, Loss: 3.945